In [2]:
import pymysql
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.ml.fpm import PrefixSpan

In [2]:
# Crwaling data from the database
db = pymysql.connect(
    host='127.0.0.1',
    user='root',
    passwd='',
    port=3306,
    db='patientdb',
    charset='utf8')
cur = db.cursor()
cur.execute(
    'SELECT MASTERPATIENTID, DISPENSECALENDARDATE, PBSDISEASEGROUP FROM nd_patient_history')
res = cur.fetchall()
df = pd.DataFrame(list(res))
df = df.rename(
    columns={
        0: 'MASTERPATIENTID',
        1: 'DISPENSECALENDARDATE',
        2: 'PBSDISEASEGROUP'})

In [7]:
# if data not from database, just use pandas to read data
df = pd.read_csv("/Users/lina/desktop/CAPSTONE/final_result.csv")
df_t = pd.read_csv("/Users/lina/desktop/CAPSTONE/medicine_result.csv")
# check if null value
df_t.GENERICINGREDIENTNAME.isnull().value_counts()
# drop null value
df_t.dropna(subset=['GENERICINGREDIENTNAME'],inplace=True)

In [44]:
# Order by name and date 
data = df_t.sort_values(
    ['MASTERPATIENTID', 'DISPENSECALENDARDATE'], ascending=[1, 1])

# drop duplicate records and leaves the first occurrence
data.drop_duplicates(
    subset=[
        'MASTERPATIENTID',
        'GENERICINGREDIENTNAME'],
    keep='first',
    inplace=True)

# Total Transactions
N = len(data)

In [170]:
def toli(disea):
    li = []
    li.append(disea)
    return li


def search_freq(disease,freqTable):
    try:
        num = int(freqTable[freqTable.itemset == disease].freq)
    except TypeError:
        num = 0
    return num


def calculate_lift(joint_count,ant_count,cons_count):
    # support(joint)/support(antecedent)*support(consequent)
    if ant_count*cons_count==0:
        num = 0
        return num
    else: 
        return (joint_count*N)/(ant_count*cons_count)


def calculate_confidence(joint_count,ant_count):
    # P(Y|X) = C(X,Y) / C(X)
    if ant_count ==0:
        num = 0
        return num
    else:
        return joint_count/ant_count

In [55]:
def prefix_rule(data,num):
    data = pd.DataFrame(data)
    # Transform to spark dataframe
    spark = SparkSession.builder.getOrCreate()
    data = spark.createDataFrame(data)
    data = data.drop('MASTERPATIENTID')
    #Create Model
    print (num)
    prefixSpan = PrefixSpan(minSupport=0.01, maxPatternLength=num,sequenceCol='GENERICINGREDIENTNAME') # 2=1:1 3=2:1
    freq = prefixSpan.findFrequentSequentialPatterns(data).sort("freq",ascending=False)
    df = freq.toPandas()
    
    return df

In [53]:
def create_table(df):
    
    AssoRule = pd.DataFrame(columns=('antecedent', 'consequent', 'joint_freq'))
    freqTable = pd.DataFrame(columns=('itemset', 'freq'))

    idx1 = 0
    idx2 = 0

    for i in range(len(df)):
    
        if len(df.loc[i]['sequence']) > 1:
            AssoRule.loc[idx1] = [df.loc[i]['sequence'][0], df.loc[i]['sequence'][1:], df.loc[i]['freq'] ]
            idx1 +=1
        
        else:
            freqTable.loc[idx2] = [df.loc[i]['sequence'][0], df.loc[i]['freq']]
            idx2 += 1
    
    return AssoRule,freqTable

In [171]:
def generate_statistics(AssoRule,freqTable):
    
    freqTable['itemset'] = freqTable['itemset'].apply(lambda x: ', '.join(each for each in x))
    AssoRule['antecedent'] = AssoRule['antecedent'].apply(lambda x: ', '.join(each for each in x))
    AssoRule['consequent'] = AssoRule['consequent'].apply(lambda x: ', '.join(each for item in x for each in item))
    AssoRule['ant_freq'] = AssoRule.apply(lambda row: search_freq(row['antecedent'],freqTable),axis=1)
    AssoRule['cons_freq'] = AssoRule.apply(lambda row: search_freq(row['consequent'],freqTable),axis=1)
    AssoRule['confidence'] = AssoRule.apply(lambda row: calculate_confidence(row['joint_freq'], row['ant_freq']), axis=1)
    AssoRule['lift'] = AssoRule.apply(lambda row: calculate_lift(row['joint_freq'], row['ant_freq'], row['cons_freq']), axis=1)
    
    return AssoRule

Confidence is a conditional probability $P(Y|X) =C(X,Y)/C(X) $ which means if a patient got disease X first, then what is the probability this patient got disease B next.

The Lift measures the probability of X and Y occurring together divided by the probability of X and Y occurring if they were independent events.That is, $P(Y|X) =P(X,Y)/P(X)*P(Y)$. If X and Y are independent then the Lift == 1. If they occur together more often than if they were independent, then Lift > 1.

## Group Concurrent Disease

In [49]:
# groupby the diseases of the same date
data = data.groupby(['MASTERPATIENTID', 'DISPENSECALENDARDATE'])['GENERICINGREDIENTNAME'].apply(list).reset_index()
# group all the records for each person in sequence
data = data.groupby(['MASTERPATIENTID'])['GENERICINGREDIENTNAME'].apply(list).reset_index()

In [172]:
#df5 = prefix_rule(data,5)
AssoRule5,freqTable5 = create_table(df5)

In [175]:
AssoRule5 = generate_statistics(AssoRule5,freqTable5)
AssoRule5.sort_values(by="confidence" , ascending=False)[1:10]

,antecedent,consequent,joint_freq,ant_freq,cons_freq,confidence,lift
1354,IRBESARTAN/HYDROCHLOROTHIAZIDE,CEFALEXIN,3914,8540,114177,0.458314,16.897280
1518,WARFARIN SODIUM,CEFALEXIN,3681,8422,114177,0.437070,16.114040
1203,SIMVASTATIN,AMOXICILLIN,4168,9803,108880,0.425176,16.438155
742,ATENOLOL,CEFALEXIN,5316,12613,114177,0.421470,15.538905
1676,IRBESARTAN/HYDROCHLOROTHIAZIDE,AMOXICILLIN,3509,8540,108880,0.410890,15.885828
673,IRBESARTAN,CEFALEXIN,5549,13649,114177,0.406550,14.988830
63,ATORVASTATIN,CEFALEXIN,13757,34543,114177,0.398257,14.683092
2115,LERCANIDIPINE,CEFALEXIN,3117,7921,114177,0.393511,14.508103
465,LEVOTHYROXINE SODIUM,CEFALEXIN,6450,16502,114177,0.390862,14.410431


In [176]:
top50 = AssoRule5.sort_values(by="confidence" , ascending=False)[1:51]

In [177]:
# output the result
outputpath="/Users/lina/desktop/CAPSTONE/top50_rules_medicine.csv"
top50.to_csv(outputpath,sep=',',index=False,header=True)